In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, Play, IntSlider, jslink, HBox
import os
from scipy.spatial.transform import Rotation as R

In [2]:
img_paths = os.listdir("/mnt/d/data/turtlebot_video/2023-5-3-1/")
img_paths = [os.path.join("/mnt/d/data/turtlebot_video/2023-5-3-1/", img_path) for img_path in img_paths]
img_paths = sorted(img_paths)
xmap = np.loadtxt('xmap_1472.pgm').astype(np.float32)
ymap = np.loadtxt('ymap_1472.pgm').astype(np.float32)

In [3]:
def spherical_project(XYZ):
    lat = np.arcsin(XYZ[..., 2])
    lon = np.arctan2(XYZ[..., 1], XYZ[..., 0])

    x = lon / np.pi
    y = lat / np.pi * 2

    xy = np.stack((x, y), axis=-1)
    return xy


def spherical_project_inverse(xy, r: float = 1.0):
    lon = np.pi * xy[..., 0]
    lat = np.pi * xy[..., 1] / 2

    XYZ = np.stack(
        (
            np.cos(lat) * np.cos(lon),
            np.cos(lat) * np.sin(lon),
            np.sin(lat),
        ),
        axis=-1,
    )
    return XYZ

In [4]:

def show(i=10):
    img = cv2.imread(img_paths[i])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w, c = img.shape
    res = cv2.remap(img, xmap.astype(np.float32), ymap.astype(np.float32), cv2.INTER_AREA, borderMode=cv2.BORDER_REPLICATE)
    x, y = np.meshgrid(np.arange(w), np.arange(h))
    x = x / w * 2 - 1
    y = y / h * 2 - 1
    xy = np.stack((x, y), axis=-1)
    xyz = spherical_project_inverse(xy)
    rot = R.from_euler('xyz', [180, 90, 0], degrees=True).as_matrix()
    xyz = xyz @ rot.T
    xy = spherical_project(xyz)
    x = xy[..., 0]
    y = xy[..., 1]
    xmap2 = (x + 1) / 2 * w
    ymap2 = (y + 1) / 2 * h
    res = cv2.remap(res, xmap2.astype(np.float32), ymap2.astype(np.float32), cv2.INTER_AREA, borderMode=cv2.BORDER_REPLICATE)
    fig = plt.figure(figsize=(10, 5))
    plt.imshow(res)
    plt.show()

play = Play(
    value=0,
    min=400,
    max=len(img_paths)-1,
    step=1,
    interval=200,
    description="Press play",
    disabled=False
)
slider = IntSlider(0, 0, len(img_paths)-1, 1)
display = interact(show, i=slider)
jslink((play, 'value'), (slider, 'value'))
HBox([play, slider])

interactive(children=(IntSlider(value=0, description='i', max=6357), Output()), _dom_classes=('widget-interact…